In [1]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_aug_1_fold8_feat.pkl'
'../features/cnn_1_aug_rescale_preprocess_feat.pkl'
'../features/cnn_1_aug_skimage_denoise_feat.pkl'
'../features/cnn_1_aug_skimage_preprocess_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_aug_1_fold8_feat.pkl'
'../features/cnn_2_aug_denoise_preprocess_feat.pkl'
'../features/cnn_2_aug_rescale_preprocess_feat.pkl'
'../features/cnn_2_aug_skimage_preprocess_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_3_aug_1_fold8_feat.pkl'
'../features/cnn_3_aug_denoise_preprocess_feat.pkl'
'../features/cnn_3_aug_rescale_preprocess_feat.pkl'
'../features/cnn_3_aug_skimage_preprocess_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_4_aug_1_fold8_feat.pkl'
'../features/cnn_4_aug_denoise_preprocess_feat.pkl'
'../features/cnn_4_aug_rescale_preprocess_feat.pkl'
'../features/cnn_4_aug_skimag

In [2]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 81) (8424, 81) (1604,)


In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=1000)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [4]:
cv_test(3,42,False)

[0]	train-logloss:0.636594	valid-logloss:0.639373
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[148]	train-logloss:0.030052	valid-logloss:0.12336

[0]	train-logloss:0.636517	valid-logloss:0.639395
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[80]	train-logloss:0.057899	valid-logloss:0.141568

[0]	train-logloss:0.637071	valid-logloss:0.639027
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[75]	train-logloss:0.060227	valid-logloss:0.143718

local average valid loss 0.137304774026 val loss std 0.00916458379182


In [5]:
cv_test(5,42,False)

[0]	train-logloss:0.635896	valid-logloss:0.639603
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[104]	train-logloss:0.047242	valid-logloss:0.14303

[0]	train-logloss:0.636493	valid-logloss:0.637751
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[134]	train-logloss:0.039225	valid-logloss:0.105996

[0]	train-logloss:0.636255	valid-logloss:0.639429
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[69]	train-logloss:0.067899	valid-logloss:0.158509

[0]	train-logloss:0.636399	valid-logloss:0.639003
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid

In [6]:
cv_test(10,42,True)

[0]	train-logloss:0.636212	valid-logloss:0.64085
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[123]	train-logloss:0.042183	valid-logloss:0.166299

[0]	train-logloss:0.636535	valid-logloss:0.638121
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[145]	train-logloss:0.036017	valid-logloss:0.114065

[0]	train-logloss:0.636596	valid-logloss:0.639044
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[118]	train-logloss:0.045574	valid-logloss:0.109725

[0]	train-logloss:0.636491	valid-logloss:0.637701
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until vali

In [7]:
cv_test(3,233,True)

[0]	train-logloss:0.637601	valid-logloss:0.63842
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[177]	train-logloss:0.024429	valid-logloss:0.109735

[0]	train-logloss:0.636143	valid-logloss:0.639263
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[125]	train-logloss:0.038698	valid-logloss:0.121327

[0]	train-logloss:0.63467	valid-logloss:0.640294
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[58]	train-logloss:0.06178	valid-logloss:0.175326

local average valid loss 0.137334707408 val loss std 0.0300850943431
         id  is_iceberg
0  5941774d    0.020480
1  4023181e    0.741906
2  b20200e4    0.032516

In [8]:
cv_test(5,233,True)

[0]	train-logloss:0.635818	valid-logloss:0.639304
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[136]	train-logloss:0.035315	valid-logloss:0.13418

[0]	train-logloss:0.636666	valid-logloss:0.634863
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[267]	train-logloss:0.014552	valid-logloss:0.069974

[0]	train-logloss:0.636293	valid-logloss:0.63801
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[109]	train-logloss:0.048782	valid-logloss:0.129253

[0]	train-logloss:0.635915	valid-logloss:0.641738
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid

In [9]:
cv_test(10,233,True)

[0]	train-logloss:0.637673	valid-logloss:0.639818
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[148]	train-logloss:0.034534	valid-logloss:0.106452

[0]	train-logloss:0.636522	valid-logloss:0.64018
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[137]	train-logloss:0.03769	valid-logloss:0.145607

[0]	train-logloss:0.636905	valid-logloss:0.636679
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[122]	train-logloss:0.047218	valid-logloss:0.075511

[0]	train-logloss:0.637077	valid-logloss:0.637841
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid